In [ ]:
# Imp libs:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Standardizing and Normalizing data:
from sklearn.preprocessing import StandardScaler

# Spliting data
from sklearn.model_selection import train_test_split, GridSearchCV

# Algorithms
from sklearn.linear_model import LinearRegression
import xgboost as xgb

# Importing metrics
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

from statsmodels.stats.outliers_influence import variance_inflation_factor

# Removing warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
df1 = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Desktop\Filp Robo worksheets and projects\flight data\dubai flight details\dtdub1.csv')
df2 = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Desktop\Filp Robo worksheets and projects\flight data\dubai flight details\dtdub2.csv')
df3 = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Desktop\Filp Robo worksheets and projects\flight data\dubai flight details\dtdub3.csv')
df4 = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Desktop\Filp Robo worksheets and projects\flight data\dubai flight details\dtdub4.csv')
df5 = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Desktop\Filp Robo worksheets and projects\flight data\dubai flight details\dtdub5.csv')
df6 = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Desktop\Filp Robo worksheets and projects\flight data\dubai flight details\dtdub6.csv')
df7 = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Desktop\Filp Robo worksheets and projects\flight data\dubai flight details\dtdub7.csv')
df8 = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Desktop\Filp Robo worksheets and projects\flight data\dubai flight details\dtdub8.csv')

list_a = [df1,df2,df3,df4,df5,df6,df7,df8]
df = pd.concat(list_a,ignore_index=True)


In [ ]:
df

In [ ]:
df = df.rename(columns = {'dept':'Departure Time','arr':'Arrival Time','ex':'Extra','stops':'Stops','prices':'Prices','airlines':'Airlines'})

df.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
df.head(5)

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
len(df['Prices'])

In [ ]:
# Feature Engineering

In [ ]:
# Prices column:

df['Prices'] = df['Prices'].str.lstrip('₹ ')

pr = []
for i in df['Prices']:
    pr.append((i.translate({ord(i): None for i in ','})))

df['Prices'] = pr

df['Prices'] = df['Prices'].astype('float64')

In [ ]:
# Total time taken by flight

t = []
for i in df['Total time taken']:
    t.append(i.split(' '))
t

time_taken_in_hours = []
for i in t:
    time_taken_in_hours.append(i[0].rstrip('h'))


df['time_taken_in_hours'] = time_taken_in_hours
df['time_taken_in_hours'] = df['time_taken_in_hours'].astype('int64')


extra_time_taken_in_minutes = []
for i in t:
    if len(i) == 1:
        extra_time_taken_in_minutes.append('0')
    else:
        extra_time_taken_in_minutes.append(i[1])

df['extra_time_taken_in_minutes'] = extra_time_taken_in_minutes
df['extra_time_taken_in_minutes'] = df['extra_time_taken_in_minutes'].astype('int64')

total_time_taken_in_minutes = df['time_taken_in_hours']*60 + df['extra_time_taken_in_minutes']

df['total_time_taken_in_hours'] = total_time_taken_in_minutes/60

In [ ]:
# Date column:

day = []
for i in df['Date']:
    day.append(i.split(',')[0])
df['Day of flight'] = day

date = []
month = []
for i in df['Date']:
    date.append(i.split(' ')[1].rstrip('th'))
    month.append(i.split(' ')[2])
    
df['date of flight'] = date
df['date of flight'] = df['date of flight'].astype('int64')

df['date of flight'] = df['date of flight'].astype('int64')
df['month of flight'] = month

In [ ]:
# Departure column:

dept= []
for i in df['Departure Time']:
    dept.append(i.replace(':','.'))
df['Departure Time'] = dept
df['Departure Time'] = df['Departure Time'].astype('float64')
art= []
for i in df['Arrival Time']:
    art.append(i.replace(':','.'))
df['Arrival Time'] = art
df['Arrival Time'] = df['Arrival Time'].astype('float64')

In [ ]:
# Dropping columns:

df.drop(columns = ['time_taken_in_hours','Total time taken','extra_time_taken_in_minutes'],axis=1,inplace=True)

df.drop(columns = ['Date'],axis=0,inplace=True)

In [ ]:
# Stop column feature engineering

stopz = []
stop_details = []
first_stop = []
second_stop = []
third_stop = []
fourth_stop = []
fifth_stop = []


for i in df['Stops']:
    stopz.append(i.split('\n'))
    
for i in stopz:
    if ',' in i:
        i.remove(',')

for i in stopz:
    if ' ' in i:
        i.remove(' ')

for i in stopz:
    stop_details.append(i[0])

for i in stopz:
    if len(i)>=2:
        first_stop.append(i[1])
    else:
        first_stop.append('NA')
    
for i in stopz:
    if len(i)>=3:
        second_stop.append(i[2])
    else:
        second_stop.append('NA')
        
        
for i in stopz:
    if len(i)>=5:
        third_stop.append(i[4])
    else:
        third_stop.append('NA')
        
for i in stopz:
    if len(i)>=7:
        fourth_stop.append(i[6])
    else:
        fourth_stop.append('NA')
        
        

df['stop_details'] = stop_details
df['first_stop'] = first_stop
df['second_stop'] = second_stop
df['third_stop'] = third_stop
df['fourth_stop'] = fourth_stop


In [ ]:
# Removing stop column

df.drop(columns=['Stops'],axis=0,inplace=True)

In [ ]:
# Airlines

airlinez = []
for i in df['Airlines']:
    if '+' not in i:
        airlinez.append(i)
    elif '+' in i:
        airlinez.append(i.split('+'))

df['Airlines'] = airlinez

airline1 = []
airline2 = []
airline3 = []


for i in airlinez:
    if len(i)==2 or len(i)==3:
        airline1.append(i[0])
    else:
        airline1.append(i)
    
for i in airlinez:
    if len(i)==2 or len(i)==3:
        airline2.append(i[1])
    else:
        airline2.append('Na')
        
for i in airlinez:
    if len(i)==3:
        airline3.append(i[2])
    else:
        airline3.append('Na')

df['airline1'] = airline1
df['airline2'] = airline2
df['airline3'] = airline3

df.drop(columns = ['Airlines'],axis=1,inplace=True)

In [ ]:
# Price range

prRange = []
for i in df['Prices']:
    if i > 12187 and i < 23554:
        prRange.append('Lower than average')
    elif i > 23554 and i < 55665:
        prRange.append('Average priced')
    else:
        prRange.append('Expensive')

df['Price Range'] = prRange 
        

In [ ]:
print('First 10 rows')
df.head(10)

In [ ]:
df.tail(20)

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
null_data = df.isnull().sum()
null_data.sort_values(ascending=0)

In [ ]:
df.columns

In [ ]:
desc = df.describe().T
desc['range']=desc['max']-desc['min']
desc

In [ ]:
df.describe(include='object').T

In [ ]:
desc[['min','mean','max','range']]

In [ ]:
cont_data = df.select_dtypes(include=['int64','float64'])

cat_data= df.select_dtypes(include=['object'])

cont_columns = cont_data.columns

cat_columns = cat_data.columns

In [ ]:
cont_columns

In [ ]:
cat_columns

In [ ]:
from scipy import stats

for i in cat_columns:
    print('For',i,', most frequent value is: ',stats.mode(df[i]),'\n')

In [ ]:
for i in cat_columns:
    print('For column',i,'unique values are: ',df[i].unique())
    print('For column',i,'count of unique values are: ',df[i].nunique(),'\n\n')

In [ ]:
for i in cat_columns:
    print('For column --',i,'-- value counts are: \n',df[i].value_counts(),'\n\n')

In [ ]:
# Univariate analysis:

In [ ]:
for i in cat_columns:
    f= plt.figure(figsize=(12,5))
    ax = sns.countplot(x=df[i],data=df)
    plt.xticks(rotation=90)

In [ ]:
# Pie chart:

for i in cat_columns:
    plt.figure(figsize=(15,15))
    df.groupby(i).size().plot(kind='pie',autopct='%.2f')
    plt.xlabel(i)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
cat_columns

In [ ]:
cont_columns

In [ ]:
df['Prices'].describe()

In [ ]:
# Univariate Summary plots

In [ ]:
# Scatterplot

for i in cont_columns:
    plt.figure(figsize=(12,8))
    sns.scatterplot(x = df.index, y = i, data = df)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
# Lineplot

for i in cont_columns:
    plt.figure(figsize=(12,8))
    sns.lineplot(x = df.index, y = i, data = df)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
# Stripplot

for i in cont_columns:
    plt.figure(figsize=(12,8))
    sns.stripplot(x = i, data = df)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
# SwarmPlot

for i in cont_columns:
    plt.figure(figsize=(12,8))
    sns.swarmplot(x = i, data = df)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
# Univariate Summary plots

In [ ]:
# Histogram plot:

for i in cont_columns:
    plt.figure(figsize=(10,6),facecolor='orange')
    plt.hist(df[i],bins=10)
    plt.xlabel(i)
    plt.show()

In [ ]:
# Densityplot

for i in cont_columns:
    plt.figure(figsize=(10,6),facecolor='orange')
    sns.kdeplot(df[i], shade=True)
    plt.xlabel(i)
    plt.show()

In [ ]:
# Boxplots

for i in cont_columns:
    plt.figure(figsize=(10,6),facecolor='orange')
    sns.boxplot(df[i])
    plt.xlabel(i)
    plt.show()

In [ ]:
# Distplot

for i in cont_columns:
    plt.figure(figsize=(10,6),facecolor='orange')
    sns.distplot(df[i])
    plt.xlabel(i)
    plt.show()

In [ ]:
# Violinplot

for i in cont_columns:
    plt.figure(figsize=(10,6),facecolor='skyblue')
    sns.violinplot(df[i],showmedians=True)
    plt.xlabel(i)
    plt.show()

In [ ]:
cont_columns

In [ ]:
# Bivariate analysis

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(x = df['Departure Time'], y = 'Prices', data = df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(x = df['Arrival Time'], y = 'Prices', data = df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(x = df['total_time_taken_in_hours'], y = 'Prices', data = df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Catplot

for i in cat_columns:
    plt.figure(figsize=(15,15))
    sns.catplot(x=i, kind="count",hue = 'Price Range', data=df)
    plt.xlabel(i)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
# Scatterplot

for i in cont_columns:
    for j in cat_columns:
            plt.figure(figsize=(12,8))
            sns.scatterplot(x = i,y=j, data = df,hue='Price Range')
            plt.xticks(rotation=90)
            plt.show()

In [ ]:
# Lineplot

for i in cont_columns:
    for j in cat_columns:
            plt.figure(figsize=(12,8))
            sns.lineplot(x = i,y=j, data = df,hue='Price Range')
            plt.xticks(rotation=90)
            plt.show()

In [ ]:
cont_columns

In [ ]:
cat_columns

In [ ]:
# Boxplots

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(x ='airline1', y = 'Departure Time', data = df, hue ='Price Range')
plt.xticks(rotation=90)
plt.show()

In [ ]:
for i in cont_columns:
    plt.figure(figsize=(15,10))
    sns.boxplot(x ='airline3', y = i, data = df, hue ='Price Range')
    plt.xticks(rotation=90)


In [ ]:
for i in cont_columns:
    plt.figure(figsize=(15,10))
    sns.boxplot(x ='airline2', y = i, data = df, hue ='Price Range')
    plt.xticks(rotation=90)


In [ ]:
for i in cont_columns:
    plt.figure(figsize=(15,10))
    sns.boxplot(x ='airline1', y = i, data = df, hue ='Price Range')
    plt.xticks(rotation=90)


In [ ]:
for i in cont_columns:
    plt.figure(figsize=(15,10))
    sns.boxplot(x ='fourth_stop', y = i, data = df, hue ='Price Range')
    plt.xticks(rotation=90)


In [ ]:
for i in cont_columns:
    plt.figure(figsize=(15,10))
    sns.boxplot(x ='third_stop', y = i, data = df, hue ='Price Range')
    plt.xticks(rotation=90)

In [ ]:
for i in cont_columns:
    plt.figure(figsize=(15,10))
    sns.boxplot(x ='second_stop', y = i, data = df, hue ='Price Range')
    plt.xticks(rotation=90)

In [ ]:
for i in cont_columns:
    plt.figure(figsize=(15,10))
    sns.boxplot(x ='first_stop', y = i, data = df, hue ='Price Range')

In [ ]:
for i in cont_columns:
    plt.figure(figsize=(15,10))
    sns.boxplot(x ='stop_details', y = i, data = df, hue ='Price Range')

In [ ]:
for i in cont_columns:
    plt.figure(figsize=(15,10))
    sns.boxplot(x ='month of flight', y = i, data = df, hue ='Price Range')

In [ ]:
for i in cont_columns:
    plt.figure(figsize=(15,10))
    sns.boxplot(x ='Day of flight', y = i, data = df, hue ='Price Range')

In [ ]:
for i in cont_columns:
    plt.figure(figsize=(15,10))
    sns.boxplot(x ='To', y = i, data = df, hue ='Price Range')

In [ ]:
for i in cont_columns:
    plt.figure(figsize=(15,10))
    sns.boxplot(x ='From', y = i, data = df, hue ='Price Range')

In [ ]:
for i in cont_columns:
    plt.figure(figsize=(15,10))
    sns.boxplot(x ='Extra', y = i, data = df, hue ='Price Range')

In [ ]:
# Swarmplot

In [ ]:
for i in cat_columns:
    plt.figure(figsize=(12,8))
    sns.swarmplot(x = i, y = "Prices", palette = "Set2", data = df)    
    plt.xlabel(i)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for i in cat_columns:
    plt.figure(figsize=(12,8))
    sns.swarmplot(x = i, y = "Price Range", palette = "Set2", data = df)    
    plt.xlabel(i)
    plt.xticks(rotation=90)
    # plt.legend()
    plt.show()


In [ ]:
cont_columns

In [ ]:
cat_columns

In [ ]:
# Scatter plot btw columns and fraud: 

for a in ['Extra', 'From', 'To', 'Day of flight', 'month of flight',
       'stop_details', 'first_stop']:
    for b in [ 'second_stop', 'third_stop',
       'fourth_stop', 'airline1', 'airline2', 'airline3']:
        sns.scatterplot(x = a, y = 'Prices',hue=b, data=df, palette='Set2')
        plt.xticks(rotation=90)
        plt.show()

In [ ]:
# Lmplot

for i in cont_columns:
    sns.lmplot(x = i, y = j,hue='Price Range', data=df, palette='Set2', height=8, aspect=1.2)
    plt.show()

In [ ]:
for i in cont_columns:
    sns.lmplot(x = i, y = 'Prices', data=df, palette='Set2', height=8, aspect=1.2)
    plt.show()

In [ ]:
# Rel plot

for i in cont_columns:
    sns.relplot(x = i, y = 'Prices', data=df,hue = 'Price Range', palette='Set2', height=8, aspect=1.2)
    plt.show()

In [ ]:
# Multivariate analysis

sns.pairplot(df)

In [ ]:
sns.pairplot(df,hue = 'Price Range')

In [ ]:
cont_columns

In [ ]:
cat_columns

In [ ]:
df['stop_details']

In [ ]:
df['stop_details'].unique()

In [ ]:
"""# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'stop_details'.
df['stop_details']= label_encoder.fit_transform(df['stop_details'])
  
df['stop_details'].unique()"""

In [ ]:
# Grouped data analysis:

In [ ]:
pd.set_option('display.max_rows', None)
df.groupby('airline1').mean().sort_values(by='Prices',ascending=False)

In [ ]:
pd.set_option('display.max_rows', None)
df.groupby('airline2').mean().sort_values(by='Prices',ascending=False)

In [ ]:
pd.set_option('display.max_rows', None)
df.groupby('airline3').mean().sort_values(by='Prices',ascending=False)

In [ ]:
pd.set_option('display.max_rows', None)
df.groupby('fourth_stop').mean().sort_values(by='Prices',ascending=False)

In [ ]:
pd.set_option('display.max_rows', None)
df.groupby('third_stop').mean().sort_values(by='Prices',ascending=False)

In [ ]:
pd.set_option('display.max_rows', None)
df.groupby('second_stop').mean().sort_values(by='Prices',ascending=False)

In [ ]:
pd.set_option('display.max_rows', None)
df.groupby('first_stop').mean().sort_values(by='Prices',ascending=False)

In [ ]:
pd.set_option('display.max_rows', None)
df.groupby('To').mean().sort_values(by='Prices',ascending=False)

In [ ]:
pd.set_option('display.max_rows', None)
df.groupby('total_time_taken_in_hours').mean().sort_values(by='Prices',ascending=False)

In [ ]:
pd.set_option('display.max_rows', None)
df.groupby('stop_details').mean().sort_values(by='Prices',ascending=False)

In [ ]:
pd.set_option('display.max_rows', None)
df.groupby('Arrival Time').mean().sort_values(by='Prices',ascending=False)

In [ ]:
pd.set_option('display.max_rows', None)
df.groupby('Departure Time').mean().sort_values(by='Prices',ascending=False)

In [ ]:
for i in cont_columns:
    for j in cat_columns:
        print('Grouped data by',j, 'on',i,':\n\n',df.groupby(j)[i].mean().sort_values(ascending = False),'\n\n')



In [ ]:
for i in cont_columns:
    for j in cat_columns:
        print('Grouped data by',j, 'on',i,':\n\n',df.groupby(j)[i].median().sort_values(ascending = False),'\n\n')

In [ ]:
# Skewness

In [ ]:
df.skew()

In [ ]:
# Correlation between features and label:

# Replacing attrition column values:

df.drop(columns = 'Prices',axis = 1).corrwith(df['Prices']).plot(kind='bar',grid=True,figsize=(10,7),title='corelation between features and labels')
plt.show()

In [ ]:
df_corr = df.corr().abs()
plt.figure(figsize=(25,22))
sns.heatmap(df_corr,annot=True,annot_kws={'size':10})
plt.show()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor


vif = pd.DataFrame()
vif['vif'] = [variance_inflation_factor(df[cont_columns],i) for i in range(df[cont_columns].shape[1])]
vif['features'] = df[cont_columns].columns
vif.sort_values(by='vif',ascending=False)

In [ ]:
cont_columns

In [ ]:
df.drop(columns=['Price Range'],axis=0,inplace=True)

In [ ]:
df1 = df.copy()

In [ ]:
# Using Z Statistics to check and remove any more outliers:

from scipy.stats import zscore

z_score = zscore(df1[cont_columns])

abs_z_score = np.abs(z_score)

filtering_entry = (abs_z_score < 3).all(axis=1) 

df1 = df1[filtering_entry]

df1.describe()

In [ ]:
df_encoded = pd.get_dummies(df)

df_encoded

In [ ]:
# Splitting data into features and label:

y = df_encoded['Prices']
X = df_encoded.drop('Prices',axis=1)

In [ ]:
# Applying standard scaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.decomposition import PCA
# Using PCA i.e. Principal Component Analysis that is a diamensionallity reduction technique:

pcaf = PCA()
pcaf.fit_transform(X_scaled)


In [ ]:
# Using Scree Plot to identify best components:

plt.figure()
plt.plot(np.cumsum(pcaf.explained_variance_ratio_))
plt.xlabel('Principal Components')
plt.ylabel('Variance Covered')
plt.title('PCA')
plt.show()


In [ ]:
from sklearn.decomposition import PCA

pcaf = PCA(n_components=110)
new_pcompf = pcaf.fit_transform(X_scaled)
princi_compf = pd.DataFrame(new_pcompf)
princi_compf

In [ ]:
# Splitting our data to training data and testing data
# x_train,x_test,y_train,y_test

x_train,x_test,y_train,y_test = train_test_split(princi_compf,y,test_size=0.30,random_state=1)

# Here we are keeping training data as our scalled data and testing data as our label or target.

In [ ]:
# MOdel instantiating and training

rm = LinearRegression()
rm.fit(x_train,y_train) 
# here we will pass training data

# Testing our model with Adjusted R2 Square: 

# on training data

print('Training dataset score: ',rm.score(x_train,y_train))

# PLotting and visualizing

y_pred = rm.predict(x_test)

plt.scatter(y_test,y_pred)
plt.xlabel('Actual Charges')
plt.ylabel('Predicted Charges')
plt.title('Actual vs Model Predicted')
plt.show()

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


#model performance

print("MAE: ", mean_absolute_error(y_test, y_pred))
print("MSE: ", mean_squared_error(y_test, y_pred))
print("RMSE: ", mean_squared_error(y_test, y_pred, squared=False))
print("R2: ", metrics.r2_score(y_test, y_pred), "\n")
print("Score: ", rm.score(x_test, y_test))

In [ ]:
from sklearn.linear_model import Ridge,Lasso,RidgeCV,LassoCV

lassocv = LassoCV(alphas = None , max_iter = 100, normalize = True)
lassocv.fit(x_train,y_train)

In [ ]:
# Best alpha parameter
alpha = lassocv.alpha_ # Best alpha rate
alpha

In [ ]:
# Now since we have the best parameter, lasso regression will be used:
lasso_reg = Lasso(alpha)
lasso_reg.fit(x_train,y_train)
# i.e. when model is training it will learn at this speed 6....

In [ ]:
lasso_reg.score(x_test,y_test)

In [ ]:
#model performance

print("MAE: ", metrics.mean_absolute_error(y_test, y_pred))
print("MSE: ", metrics.mean_squared_error(y_test, y_pred))
print("RMSE: ", metrics.mean_squared_error(y_test, y_pred, squared=False))
print("R2: ", metrics.r2_score(y_test, y_pred), "\n")
print("Score: ", lasso_reg.score(x_test, y_test))

In [ ]:
# Ridge Method:

ridgecv = RidgeCV(alphas = np.arange(0.001,0.1,0.01),normalize = True)
ridgecv.fit(x_train,y_train)

In [ ]:
ridgecv.alpha_ # Best alpha rate

In [ ]:
ridge_model = Ridge(alpha = ridgecv.alpha_)
ridge_model.fit(x_train,y_train)

In [ ]:
ridge_model.score(x_test,y_test)

In [ ]:
#model performance

print("MAE: ", metrics.mean_absolute_error(y_test, y_pred))
print("MSE: ", metrics.mean_squared_error(y_test, y_pred))
print("RMSE: ", metrics.mean_squared_error(y_test, y_pred, squared=False))
print("R2: ", metrics.r2_score(y_test, y_pred), "\n")
print("Score: ", ridge_model.score(x_test, y_test))

In [ ]:
# Using decision tree:

from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import r2_score

model = DecisionTreeRegressor()

model.fit(x_train, y_train)

y_preddt = model.predict(x_test)

r2_score(y_test,y_preddt)


In [ ]:
# Random forest:

from sklearn.ensemble import RandomForestRegressor
regressor_rf = RandomForestRegressor(n_estimators = 200)

regressor_rf.fit(x_train, y_train)

lr_normal_rf = regressor_rf.score(x_train, y_train)

lr_normal_rf

In [ ]:
y_predrf = regressor_rf.predict(x_test)

lr_normal_rf_test = regressor_rf.score(x_test, y_test)

lr_normal_rf_test

mse_lr_normal_rf  = mean_absolute_error(y_test, y_predrf)

mse_lr_normal_rf

In [ ]:
# Model Evaluation: MAE , MSE , RMSE

from sklearn.metrics import mean_absolute_error,mean_squared_error

print("MAE: ", metrics.mean_absolute_error(y_test, y_predrf))
print("MSE: ", metrics.mean_squared_error(y_test, y_predrf))
print("RMSE: ", metrics.mean_squared_error(y_test, y_predrf, squared=False))
print("R2: ", metrics.r2_score(y_test, y_predrf), "\n")
print("Score: ", regressor_rf.score(x_test, y_predrf))

In [ ]:
from sklearn.model_selection import KFold , cross_val_score
k_f = KFold(n_splits =3)
k_f

# Cross Validation score to check if model is overfitting:

cross_val_score(regressor_rf,x_train,y_train,cv=3) # 

In [ ]:
# Tuning parameters using GridSearchCV:

params = {'n_estimators':[100],
          'max_depth':[10,20],
          'min_samples_split':[11,12],
          'min_samples_leaf':[5,6]}

grd = GridSearchCV(regressor_rf,param_grid=params)

grd.fit(x_train,y_train)

In [ ]:
# Getting best parameters:

print('Best parameters: ',grd.best_params_)

In [ ]:
# Reinstantiating:

rf_boosted = RandomForestRegressor(n_estimators=100,max_depth=20,
                                    min_samples_split=11,min_samples_leaf=5)
                                    
rf_boosted.fit(x_train, y_train)

rf_boo_score = rf_boosted.score(x_train, y_train)

print('Training score: ',rf_boo_score)

print('Testing score: ',rf_boosted.score(x_test,y_test))

y_predrfb = rf_boosted.predict(x_test)

print("MAE: ", metrics.mean_absolute_error(y_test, y_predrfb))
print("MSE: ", metrics.mean_squared_error(y_test, y_predrfb))
print("RMSE: ", metrics.mean_squared_error(y_test, y_predrfb, squared=False))
print("R2: ", metrics.r2_score(y_test, y_predrfb), "\n")
print("Score: ", rf_boosted.score(x_test, y_predrfb))

In [ ]:
from sklearn.model_selection import KFold , cross_val_score
k_f = KFold(n_splits =3)
k_f

# Cross Validation score to check if model is overfitting:

cross_val_score(rf_boosted,x_train,y_train,cv=5) # 

In [ ]:
# Using Support vector regressor:

# Fit the model over the training data
from sklearn.svm import SVR
svr = SVR()
svr.fit(x_train, y_train)

y_predsvr = svr.predict(x_test)

r2_score(y_test,y_predsvr)

In [ ]:
"""# Finding best random state for logistic regression model:

maxscore = 0
maxRS = 0

for i in range(1,200):
    rf_boosted = RandomForestRegressor(n_estimators=100,max_depth=10,
                                    min_samples_split=11,min_samples_leaf=5,random_state=i)
    rf_boosted.fit(x_train, y_train) 
    y_pred = rf_boosted.predict(x_test)
    score = r2_score(y_test,y_pred)
    print('Testing accuracy: ', score, 'at random state', i)
    
    if score > maxscore:
        maxscore = score
        maxRS = i
        print('Max score',maxscore,'max random state',maxRS)"""

In [ ]:
# Using XGBoost:

xgb_reg = xgb.XGBRegressor()
xgb_reg.fit(x_train,y_train)
xgb_reg_score = xgb_reg.score(x_train, y_train)

print('Training score: ',xgb_reg.score(x_train, y_train))

print('Testing score: ',xgb_reg.score(x_test,y_test))

y_predxb = xgb_reg.predict(x_test)

print("MAE: ", metrics.mean_absolute_error(y_test, y_predxb))
print("MSE: ", metrics.mean_squared_error(y_test, y_predxb))
print("RMSE: ", metrics.mean_squared_error(y_test, y_predxb, squared=False))
print("R2: ", metrics.r2_score(y_test, y_predxb), "\n")
print("Score: ", xgb_reg.score(x_test, y_predxb))

r2_score(y_test,y_predxb)

In [ ]:
from sklearn.model_selection import KFold , cross_val_score
k_f = KFold(n_splits =3)
k_f

# Cross Validation score to check if model is overfitting:

cross_val_score(xgb_reg,x_train,y_train,cv=5) # 

In [ ]:
# Hyperparameter tuning for xgboost model

params = {"learning_rate"    : [0.10,0.2,0.3] ,
         "max_depth"        : [ 1,2,3]}

grd = GridSearchCV(xgb_reg,param_grid=params)

xgb_boosted = grd.fit(x_train,y_train)

y_predxx = xgb_boosted.predict(x_test)

r2_score(y_test,y_predxx)

In [ ]:
grd.best_params_

In [ ]:
xgb_boosted = xgb.XGBRegressor(max_depth=3,learning_rate=0.2,n_estimators=200)

xgb_boosted.fit(x_train,y_train)

y_predxx = xgb_boosted.predict(x_test)

r2_score(y_test,y_predxx)

In [ ]:
# Model Evaluation: MAE , MSE , RMSE

from sklearn.metrics import mean_absolute_error,mean_squared_error

print("MAE: ", metrics.mean_absolute_error(y_test, y_predxx))
print("MSE: ", metrics.mean_squared_error(y_test, y_predxx))
print("RMSE: ", metrics.mean_squared_error(y_test, y_predxx, squared=False))
print("R2: ", metrics.r2_score(y_test, y_predxx), "\n")
print("Score: ", xgb_boosted.score(x_test, y_predxx))

In [ ]:
from sklearn.model_selection import KFold , cross_val_score
k_f = KFold(n_splits =3)
k_f

# Cross Validation score to check if model is overfitting:

cross_val_score(xgb_boosted,x_train,y_train,cv=5) # 

In [ ]:
# Innitiate k neighbour Regressor:

from sklearn.neighbors import KNeighborsRegressor

params = {'n_neighbors':[0,1,2,3,4,5,6,7,8,9]}

knn = KNeighborsRegressor()

knnmodel = GridSearchCV(knn, params, cv=5)
knnmodel.fit(x_train,y_train)
knnmodel.best_params_

In [ ]:
knn_best = KNeighborsRegressor(n_neighbors = 1)
knn_best.fit(x_train,y_train)

In [ ]:
y_predknn = knn_best.predict(x_test)

r2_score(y_test,y_predknn)

In [ ]:
# Adaboost

from sklearn.ensemble import AdaBoostRegressor

regr = AdaBoostRegressor(random_state=0, n_estimators=100)

regr.fit(x_train,y_train)

ypredada = regr.predict(x_test)

r2_score(y_test,ypredada)

In [ ]:
# Model Evaluation: MAE , MSE , RMSE

from sklearn.metrics import mean_absolute_error,mean_squared_error

print("MAE: ", metrics.mean_absolute_error(y_test, ypredada))
print("MSE: ", metrics.mean_squared_error(y_test, ypredada))
print("RMSE: ", metrics.mean_squared_error(y_test, ypredada, squared=False))
print("R2: ", metrics.r2_score(y_test, ypredada), "\n")
print("Score: ", regr.score(x_test, ypredada))

In [ ]:
# Gradient Boosting regressor

from sklearn.ensemble import GradientBoostingRegressor

reg = GradientBoostingRegressor(random_state=0)
reg.fit(x_train,y_train)

ypredgbr = regr.predict(x_test)

r2_score(y_test,ypredgbr)

In [ ]:
# Model Evaluation: MAE , MSE , RMSE

from sklearn.metrics import mean_absolute_error,mean_squared_error

print("MAE: ", metrics.mean_absolute_error(y_test, ypredgbr))
print("MSE: ", metrics.mean_squared_error(y_test, ypredgbr))
print("RMSE: ", metrics.mean_squared_error(y_test, ypredgbr, squared=False))
print("R2: ", metrics.r2_score(y_test, ypredgbr), "\n")
print("Score: ", reg.score(x_test, ypredgbr))

In [ ]:
from sklearn.model_selection import KFold , cross_val_score
k_f = KFold(n_splits =3)
k_f

# Cross Validation score to check if model is overfitting:

cross_val_score(reg,x_train,y_train,cv=5) # 

In [ ]:
# Creating pipeline:
from sklearn import metrics

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error,mean_squared_error



from sklearn.pipeline import Pipeline

pipe1 = Pipeline([('scaler',StandardScaler()),
                  ('pca',PCA(n_components=110)),('base_model1',RandomForestRegressor(n_estimators=100,max_depth=20,
                                    min_samples_split=11,min_samples_leaf=5))])

pipe1.fit(x_train,y_train)

y_pred = pipe1.predict(x_test)

print("MAE: ", metrics.mean_absolute_error(y_test, y_pred))
print("MSE: ", metrics.mean_squared_error(y_test, y_pred))
print("RMSE: ", metrics.mean_squared_error(y_test, y_pred, squared=False))
print("R2: ", metrics.r2_score(y_test, y_pred), "\n")
print("Score: ", pipe1.score(x_test, y_pred))


# Saving regression model to pickle string


import pickle 
saved_model1 = pickle.dumps(pipe1) 
pipe_pickle1 = pickle.loads(saved_model1)
pipe_pickle1.predict(x_test) # predicting testing data

